1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

## Setup

### Install & import

In [11]:
!pip install --upgrade google-cloud-aiplatform

In [12]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
import json

## Parse the arguments

2. Use the following code in your application to request a model response

In [65]:

# model = GenerativeModel("projects/714199212179/locations/us-central1/endpoints/865067161431834624") # vertex-sdk-large
# model = GenerativeModel("projects/714199212179/locations/us-central1/endpoints/4596017977731579904") # vertex-sdk-small
# model = GenerativeModel("projects/714199212179/locations/us-central1/endpoints/8446595659133353984") # vertex-studio-large
model = GenerativeModel("projects/714199212179/locations/us-central1/endpoints/993138275835183104") # vertex-studio-small

generation_config = {
    "max_output_tokens": 2048,
    "temperature": 1,
    "top_p": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [66]:
print('Model: ', model._model_name)
print('Generation config:', json.dumps(generation_config, indent=2))
print('Safety settings: ', json.dumps(safety_settings, indent=2))

Model:  projects/714199212179/locations/us-central1/endpoints/993138275835183104
Generation config: {
  "max_output_tokens": 2048,
  "temperature": 1,
  "top_p": 1
}
Safety settings:  {
  "1": 2,
  "2": 2,
  "4": 2,
  "3": 2
}


In [55]:
# Import test dataset from local csv or jsonl file

import pandas as pd
data = pd.read_csv("file://localhost/content/dialogsum.test.truncated.csv")
data = data.drop(columns=['topic1','summary2','topic2','summary3','topic3'])
data

,fname,dialogue,summary1
0,test_0,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...
1,test_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....
2,test_2,"#Person1#: Kate, you never believe what's happ...",#Person1# tells Kate that Masha and Hero get d...
3,test_3,"#Person1#: Happy Birthday, this is for you, Br...",#Person1# and Brian are at the birthday party ...
4,test_4,#Person1#: This Olympic park is so big!\n#Pers...,#Person1# is surprised at the Olympic Stadium'...
...,...,...,...
95,test_95,#Person1#: Thank you. Steven. That was the mos...,Steven and Lin just had a great meal. Then the...
96,test_96,"#Person1#: Bill, how can you hear so happy tod...",Bill is happy because he made a move to know h...
97,test_97,#Person1#: I'd like to pay my bill now. \n#Per...,#Person2# checks Tom Wilson's information and ...
98,test_98,#Person1#: Carol telephone.\n#Person2#: Who is...,Susan calls Carol to ask about the party time....


In [56]:
# format testing data


testing_data = {}
# {'test_n': 'dialogue'}
for _, row in data.iterrows():
  testing_data[row['fname']] = row['dialogue']

# for fname in testing_data:
#   print(fname)

testing_data

{'test_0': "#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please contin

## Call the API

In [67]:
vertexai.init(project="dane-0424", location="us-central1")

responses = []
results = {}

for fname in testing_data:
  dialogue = testing_data[fname]
  response = model.generate_content(
      [dialogue],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )
  responses.append(response)

  if response.candidates[0].finish_reason == 1:
    results[fname] = {'dialogue': dialogue, 'summary': response.text}
  else:
    results[fname] = {'dialogue': dialogue, 'summary': response.candidates[0].finish_reason }

In [68]:
for response in responses:
  print(response.candidates)


[content {
  role: "model"
  parts {
    text: "Ms. Dawson takes a dictation for #Person1#. #Person1# makes a new policy which prohibits the employees using Instant Messaging during working hours."
  }
}
finish_reason: STOP
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
  probability_score: 0.1677633374929428
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0.0697949230670929
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
  probability_score: 0.28987398743629456
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0.10970578342676163
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
  probability_score: 0.2660711407661438
  severity: HARM_SEVERITY_LOW
  severity_score: 0.20561504364013672
}
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
  probability_score: 0.19713513553142548
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0

In [70]:
# Export results to local csv file

df = pd.DataFrame.from_dict(results, orient='index')
df.index.name = 'fname'
df = df.reset_index()
df
df.to_csv('tuned.vertex.studio.small.output.csv', index=False)